In [14]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

True
1
0
GeForce RTX 3080 Ti


In [101]:
import numpy as np

from tensorforce import Agent, Environment, Runner


class MultiactorEnvironment(Environment):
    """
    Example multi-actor environment, illustrating best-practice implementation pattern.
    State space: position in [0, 10].
    Action space: movement in {-1, 0, 1}.
    Random start in [3, 7].
    Actor 1 perspective as is, actor 2 perspective mirrored.
    Positive reward for being closer to 10.
    """

    def __init__(self):
        super().__init__()

    def states(self):
        return dict(type='int', num_values=11)

    def actions(self):
        return dict(type='int', num_values=3)

    def num_actors(self):
        return 1  # Indicates that environment has multiple actors

    def reset(self, num_parallel = 1):
        # Always for multi-actor environments: initialize parallel indices
        self._parallel_indices = np.arange(num_parallel)

        # Single shared environment logic, plus per-actor perspective
        self._states = 3 + np.random.randint(5)
        self.second_actor = True
        states = np.stack([self._states, 10 - self._states], axis=0)

        # Always for multi-actor environments: return per-actor values
        return self._parallel_indices.copy(), states

    def execute(self, actions):
        # Single shared environment logic, plus per-actor perspective
        if self.second_actor:
            self.second_actor = self.second_actor and not (np.random.random_sample() < 0.1)
            terminal = np.stack([False, False], axis=0)
            delta = (actions[0] - 1) - (actions[1] - 1)
            rand_val = np.clip(self._states + delta, a_min=0, a_max=10)
            states = np.stack([self._states, rand_val], axis=0)
        else:
            terminal = np.stack([False, False], axis=0)
            delta = (actions[0] - 1)
            self._states = np.clip(self._states + delta, a_min=0, a_max=10)
            states = np.stack([self._states, self._states], axis=0)
        reward = np.stack([self._states, 10 - self._states], axis=0)

        # Always for multi-actor environments: update parallel indices, and return per-actor values
        self._parallel_indices = self._parallel_indices[~terminal]
        return self._parallel_indices.copy(), states, terminal, reward
    
    def is_vectorizable(self):
        return True

environment = Environment.create(
    environment=MultiactorEnvironment, max_episode_timesteps=10000
)

num_parallel = 2
agent = Agent.create(
    agent='ppo',
    environment=environment,  # alternatively: states, actions, (max_episode_timesteps)
    network={"type": "auto", "rnn": False},
    use_beta_distribution=False,
    memory="minimum",
    batch_size=12,
    update_frequency=1,
    learning_rate=0.001813150053725916,
    multi_step=5,
    subsampling_fraction=0.9131375430837279,
    likelihood_ratio_clipping=0.09955676846552193,
    discount=0.9985351346308641,
    return_processing=None,
    advantage_processing=None,
    predict_terminal_values=False,
    reward_processing=None,
    baseline={"type": "auto", "rnn":False},
    baseline_optimizer={"optimizer": "adam", "learning_rate": 0.003670157218888348, "multi_step":10},
    l2_regularization=0.0,
    entropy_regularization=0.0011393096635237982,
    state_preprocessing="linear_normalization",
    exploration=0.0,
    variable_noise=0.0,
    parallel_interactions=num_parallel
)

In [102]:
# Train for 300 episodes
for _ in range(2):
    # Initialize episode
    parallel_index, states = environment.reset(num_parallel=num_parallel)
    terminal = [False]*num_parallel
    while not terminal[0]:
        # Episode timestep
        actions = agent.act(parallel=parallel_index, states=states)
        next_parallel, states, terminal, reward = environment.execute(actions=actions)
        
        print("iterx")
        print(next_parallel)
        print(states)
        print(actions)
        print(terminal)
        
        agent.observe(terminal=terminal, reward=reward, parallel=parallel_index)
        
        '''
        actions = agent2.act(states=states)
        states, terminal, reward = environment.execute(actions=actions)
        agent2.observe(terminal=terminal, reward=reward)
        print(states)
        print(actions)
        '''
        
# agent.close()
# environment.close()

iterx
[0 1]
[7 9]
[2, 0]
[0 0]
iterx
[0 1]
[7 6]
[0, 1]
[0 0]
iterx
[0 1]
[7 7]
[2, 2]
[0 0]
iterx
[0 1]
[7 9]
[2, 0]
[0 0]
iterx
[0 1]
[7 7]
[1, 1]
[0 0]
iterx
[0 1]
[6 6]
[0, 0]
[0 0]
iterx
[0 1]
[5 5]
[0, 2]
[0 0]
iterx
[0 1]
[6 6]
[2, 2]
[0 0]
iterx
[0 1]
[5 5]
[0, 2]
[0 0]
iterx
[0 1]
[6 6]
[2, 0]
[0 0]
iterx
[0 1]
[5 5]
[0, 1]
[0 0]
iterx
[0 1]
[5 5]
[1, 1]
[0 0]
iterx
[0 1]
[4 4]
[0, 0]
[0 0]
iterx
[0 1]
[4 4]
[1, 1]
[0 0]
iterx
[0 1]
[5 5]
[2, 2]
[0 0]
iterx
[0 1]
[6 6]
[2, 1]
[0 0]
iterx
[0 1]
[7 7]
[2, 0]
[0 0]
iterx
[0 1]
[6 6]
[0, 2]
[0 0]
iterx
[0 1]
[5 5]
[0, 0]
[0 0]
iterx
[0 1]
[4 4]
[0, 2]
[0 0]
iterx
[0 1]
[3 3]
[0, 1]
[0 0]
iterx
[0 1]
[2 2]
[0, 2]
[0 0]
iterx
[0 1]
[3 3]
[2, 2]
[0 0]
iterx
[0 1]
[4 4]
[2, 1]
[0 0]
iterx
[0 1]
[5 5]
[2, 0]
[0 0]
iterx
[0 1]
[6 6]
[2, 1]
[0 0]
iterx
[0 1]
[5 5]
[0, 0]
[0 0]
iterx
[0 1]
[4 4]
[0, 1]
[0 0]
iterx
[0 1]
[5 5]
[2, 1]
[0 0]
iterx
[0 1]
[4 4]
[0, 0]
[0 0]
iterx
[0 1]
[5 5]
[2, 0]
[0 0]
iterx
[0 1]
[6 6]
[2, 1]
[0 0]
iterx
[0

iterx
[0 1]
[8 8]
[1, 1]
[0 0]
iterx
[0 1]
[7 7]
[0, 0]
[0 0]
iterx
[0 1]
[6 6]
[0, 2]
[0 0]
iterx
[0 1]
[5 5]
[0, 1]
[0 0]
iterx
[0 1]
[6 6]
[2, 1]
[0 0]
iterx
[0 1]
[7 7]
[2, 1]
[0 0]
iterx
[0 1]
[8 8]
[2, 0]
[0 0]
iterx
[0 1]
[8 8]
[1, 1]
[0 0]
iterx
[0 1]
[9 9]
[2, 1]
[0 0]
iterx
[0 1]
[10 10]
[2, 1]
[0 0]
iterx
[0 1]
[9 9]
[0, 1]
[0 0]
iterx
[0 1]
[8 8]
[0, 0]
[0 0]
iterx
[0 1]
[8 8]
[1, 2]
[0 0]
iterx
[0 1]
[9 9]
[2, 2]
[0 0]
iterx
[0 1]
[9 9]
[1, 1]
[0 0]
iterx
[0 1]
[9 9]
[1, 2]
[0 0]
iterx
[0 1]
[10 10]
[2, 1]
[0 0]
iterx
[0 1]
[10 10]
[2, 1]
[0 0]
iterx
[0 1]
[10 10]
[2, 0]
[0 0]
iterx
[0 1]
[9 9]
[0, 1]
[0 0]
iterx
[0 1]
[8 8]
[0, 1]
[0 0]
iterx
[0 1]
[8 8]
[1, 1]
[0 0]
iterx
[0 1]
[9 9]
[2, 1]
[0 0]
iterx
[0 1]
[9 9]
[1, 0]
[0 0]
iterx
[0 1]
[10 10]
[2, 0]
[0 0]
iterx
[0 1]
[10 10]
[1, 2]
[0 0]
iterx
[0 1]
[9 9]
[0, 2]
[0 0]
iterx
[0 1]
[9 9]
[1, 1]
[0 0]
iterx
[0 1]
[8 8]
[0, 1]
[0 0]
iterx
[0 1]
[8 8]
[1, 1]
[0 0]
iterx
[0 1]
[9 9]
[2, 2]
[0 0]
iterx
[0 1]
[9 9]
[1, 2]
[0

iterx
[0 1]
[7 7]
[0, 1]
[0 0]
iterx
[0 1]
[8 8]
[2, 0]
[0 0]
iterx
[0 1]
[9 9]
[2, 2]
[0 0]
iterx
[0 1]
[9 9]
[1, 1]
[0 0]
iterx
[0 1]
[8 8]
[0, 2]
[0 0]
iterx
[0 1]
[7 7]
[0, 2]
[0 0]
iterx
[0 1]
[6 6]
[0, 1]
[0 0]
iterx
[0 1]
[5 5]
[0, 2]
[0 0]
iterx
[0 1]
[4 4]
[0, 0]
[0 0]
iterx
[0 1]
[5 5]
[2, 1]
[0 0]
iterx
[0 1]
[6 6]
[2, 2]
[0 0]
iterx
[0 1]
[6 6]
[1, 1]
[0 0]
iterx
[0 1]
[5 5]
[0, 1]
[0 0]
iterx
[0 1]
[6 6]
[2, 1]
[0 0]
iterx
[0 1]
[6 6]
[1, 0]
[0 0]
iterx
[0 1]
[5 5]
[0, 1]
[0 0]
iterx
[0 1]
[4 4]
[0, 0]
[0 0]
iterx
[0 1]
[4 4]
[1, 1]
[0 0]
iterx
[0 1]
[5 5]
[2, 0]
[0 0]
iterx
[0 1]
[6 6]
[2, 2]
[0 0]
iterx
[0 1]
[6 6]
[1, 0]
[0 0]
iterx
[0 1]
[7 7]
[2, 1]
[0 0]
iterx
[0 1]
[7 7]
[1, 1]
[0 0]
iterx
[0 1]
[6 6]
[0, 0]
[0 0]
iterx
[0 1]
[6 6]
[1, 0]
[0 0]
iterx
[0 1]
[5 5]
[0, 0]
[0 0]
iterx
[0 1]
[4 4]
[0, 0]
[0 0]
iterx
[0 1]
[3 3]
[0, 1]
[0 0]
iterx
[0 1]
[2 2]
[0, 0]
[0 0]
iterx
[0 1]
[3 3]
[2, 0]
[0 0]
iterx
[0 1]
[2 2]
[0, 2]
[0 0]
iterx
[0 1]
[3 3]
[2, 1]
[0 0]
iterx
[0

iterx
[0 1]
[10 10]
[2, 0]
[0 0]
iterx
[0 1]
[10 10]
[1, 2]
[0 0]
iterx
[0 1]
[10 10]
[1, 2]
[0 0]
iterx
[0 1]
[9 9]
[0, 1]
[0 0]
iterx
[0 1]
[9 9]
[1, 2]
[0 0]
iterx
[0 1]
[9 9]
[1, 0]
[0 0]
iterx
[0 1]
[9 9]
[1, 0]
[0 0]
iterx
[0 1]
[10 10]
[2, 1]
[0 0]
iterx
[0 1]
[10 10]
[1, 2]
[0 0]
iterx
[0 1]
[9 9]
[0, 2]
[0 0]
iterx
[0 1]
[10 10]
[2, 2]
[0 0]
iterx
[0 1]
[10 10]
[2, 1]
[0 0]
iterx
[0 1]
[9 9]
[0, 2]
[0 0]
iterx
[0 1]
[10 10]
[2, 1]
[0 0]
iterx
[0 1]
[9 9]
[0, 2]
[0 0]
iterx
[0 1]
[10 10]
[2, 0]
[0 0]
iterx
[0 1]
[10 10]
[1, 0]
[0 0]
iterx
[0 1]
[10 10]
[2, 2]
[0 0]
iterx
[0 1]
[10 10]
[1, 1]
[0 0]
iterx
[0 1]
[10 10]
[2, 2]
[0 0]
iterx
[0 1]
[9 9]
[0, 2]
[0 0]
iterx
[0 1]
[10 10]
[2, 0]
[0 0]
iterx
[0 1]
[10 10]
[1, 1]
[0 0]
iterx
[0 1]
[10 10]
[2, 1]
[0 0]
iterx
[0 1]
[10 10]
[1, 0]
[0 0]
iterx
[0 1]
[10 10]
[1, 1]
[0 0]
iterx
[0 1]
[10 10]
[1, 2]
[0 0]
iterx
[0 1]
[9 9]
[0, 2]
[0 0]
iterx
[0 1]
[10 10]
[2, 1]
[0 0]
iterx
[0 1]
[10 10]
[1, 1]
[0 0]
iterx
[0 1]
[10 10]
[2, 0]
[

iterx
[0 1]
[3 3]
[0, 0]
[0 0]
iterx
[0 1]
[2 2]
[0, 0]
[0 0]
iterx
[0 1]
[2 2]
[1, 0]
[0 0]
iterx
[0 1]
[1 1]
[0, 1]
[0 0]
iterx
[0 1]
[0 0]
[0, 1]
[0 0]
iterx
[0 1]
[0 0]
[1, 2]
[0 0]
iterx
[0 1]
[1 1]
[2, 0]
[0 0]
iterx
[0 1]
[1 1]
[1, 1]
[0 0]
iterx
[0 1]
[1 1]
[1, 0]
[0 0]
iterx
[0 1]
[2 2]
[2, 0]
[0 0]
iterx
[0 1]
[1 1]
[0, 1]
[0 0]
iterx
[0 1]
[1 1]
[1, 0]
[0 0]
iterx
[0 1]
[2 2]
[2, 2]
[0 0]
iterx
[0 1]
[3 3]
[2, 2]
[0 0]
iterx
[0 1]
[2 2]
[0, 2]
[0 0]
iterx
[0 1]
[2 2]
[1, 1]
[0 0]
iterx
[0 1]
[2 2]
[1, 0]
[0 0]
iterx
[0 1]
[1 1]
[0, 0]
[0 0]
iterx
[0 1]
[1 1]
[1, 2]
[0 0]
iterx
[0 1]
[2 2]
[2, 1]
[0 0]
iterx
[0 1]
[1 1]
[0, 0]
[0 0]
iterx
[0 1]
[0 0]
[0, 2]
[0 0]
iterx
[0 1]
[1 1]
[2, 2]
[0 0]
iterx
[0 1]
[2 2]
[2, 0]
[0 0]
iterx
[0 1]
[1 1]
[0, 1]
[0 0]
iterx
[0 1]
[2 2]
[2, 1]
[0 0]
iterx
[0 1]
[1 1]
[0, 1]
[0 0]
iterx
[0 1]
[0 0]
[0, 1]
[0 0]
iterx
[0 1]
[0 0]
[0, 2]
[0 0]
iterx
[0 1]
[0 0]
[0, 0]
[0 0]
iterx
[0 1]
[1 1]
[2, 1]
[0 0]
iterx
[0 1]
[1 1]
[1, 0]
[0 0]
iterx
[0

...

iterx
[0 1]
[10 10]
[2, 2]
[0 0]
iterx
[0 1]
[10 10]
[1, 2]
[0 0]
iterx
[0 1]
[10 10]
[1, 2]
[0 0]
iterx
[0 1]
[9 9]
[0, 0]
[0 0]
iterx
[0 1]
[10 10]
[2, 0]
[0 0]
iterx
[0 1]
[10 10]
[2, 0]
[0 0]
iterx
[0 1]
[10 10]
[1, 0]
[0 0]
iterx
[0 1]
[9 9]
[0, 2]
[0 0]
iterx
[0 1]
[10 10]
[2, 2]
[0 0]
iterx
[0 1]
[10 10]
[1, 1]
[0 0]
iterx
[0 1]
[10 10]
[1, 0]
[0 0]
iterx
[0 1]
[10 10]
[2, 2]
[0 0]
iterx
[0 1]
[10 10]
[1, 1]
[0 0]
iterx
[0 1]
[10 10]
[1, 2]
[0 0]
iterx
[0 1]
[10 10]
[1, 1]
[0 0]
iterx
[0 1]
[10 10]
[1, 2]
[0 0]
iterx
[0 1]
[9 9]
[0, 1]
[0 0]
iterx
[0 1]
[10 10]
[2, 1]
[0 0]
iterx
[0 1]
[9 9]
[0, 1]
[0 0]
iterx
[0 1]
[10 10]
[2, 1]
[0 0]
iterx
[0 1]
[9 9]
[0, 2]
[0 0]
iterx
[0 1]
[8 8]
[0, 2]
[0 0]
iterx
[0 1]
[9 9]
[2, 2]
[0 0]
iterx
[0 1]
[9 9]
[1, 2]
[0 0]
iterx
[0 1]
[8 8]
[0, 1]
[0 0]
iterx
[0 1]
[8 8]
[1, 0]
[0 0]
iterx
[0 1]
[7 7]
[0, 2]
[0 0]
iterx
[0 1]
[6 6]
[0, 2]
[0 0]
iterx
[0 1]
[5 5]
[0, 0]
[0 0]
iterx
[0 1]
[5 5]
[1, 0]
[0 0]
iterx
[0 1]
[4 4]
[0, 0]
[0 0]
iterx
[

iterx
[0 1]
[6 6]
[2, 0]
[0 0]
iterx
[0 1]
[5 5]
[0, 0]
[0 0]
iterx
[0 1]
[6 6]
[2, 0]
[0 0]
iterx
[0 1]
[5 5]
[0, 1]
[0 0]
iterx
[0 1]
[6 6]
[2, 1]
[0 0]
iterx
[0 1]
[6 6]
[1, 0]
[0 0]
iterx
[0 1]
[6 6]
[1, 0]
[0 0]
iterx
[0 1]
[6 6]
[1, 1]
[0 0]
iterx
[0 1]
[6 6]
[1, 0]
[0 0]
iterx
[0 1]
[5 5]
[0, 1]
[0 0]
iterx
[0 1]
[5 5]
[1, 1]
[0 0]
iterx
[0 1]
[6 6]
[2, 0]
[0 0]
iterx
[0 1]
[6 6]
[1, 0]
[0 0]
iterx
[0 1]
[7 7]
[2, 2]
[0 0]
iterx
[0 1]
[8 8]
[2, 1]
[0 0]
iterx
[0 1]
[7 7]
[0, 0]
[0 0]
iterx
[0 1]
[7 7]
[1, 0]
[0 0]
iterx
[0 1]
[7 7]
[1, 1]
[0 0]
iterx
[0 1]
[7 7]
[1, 1]
[0 0]
iterx
[0 1]
[7 7]
[1, 0]
[0 0]
iterx
[0 1]
[6 6]
[0, 0]
[0 0]
iterx
[0 1]
[6 6]
[1, 1]
[0 0]
iterx
[0 1]
[5 5]
[0, 0]
[0 0]
iterx
[0 1]
[4 4]
[0, 1]
[0 0]
iterx
[0 1]
[5 5]
[2, 0]
[0 0]
iterx
[0 1]
[6 6]
[2, 2]
[0 0]
iterx
[0 1]
[5 5]
[0, 2]
[0 0]
iterx
[0 1]
[5 5]
[1, 1]
[0 0]
iterx
[0 1]
[4 4]
[0, 2]
[0 0]
iterx
[0 1]
[4 4]
[1, 2]
[0 0]
iterx
[0 1]
[5 5]
[2, 2]
[0 0]
iterx
[0 1]
[4 4]
[0, 0]
[0 0]
iterx
[0

iterx
[0 1]
[1 1]
[0, 0]
[0 0]
iterx
[0 1]
[1 1]
[1, 2]
[0 0]
iterx
[0 1]
[1 1]
[1, 1]
[0 0]
iterx
[0 1]
[1 1]
[1, 1]
[0 0]
iterx
[0 1]
[0 0]
[0, 0]
[0 0]
iterx
[0 1]
[1 1]
[2, 0]
[0 0]
iterx
[0 1]
[1 1]
[1, 0]
[0 0]
iterx
[0 1]
[0 0]
[0, 0]
[0 0]
iterx
[0 1]
[0 0]
[1, 1]
[0 0]
iterx
[0 1]
[0 0]
[1, 1]
[0 0]
iterx
[0 1]
[1 1]
[2, 2]
[0 0]
iterx
[0 1]
[0 0]
[0, 2]
[0 0]
iterx
[0 1]
[0 0]
[0, 2]
[0 0]
iterx
[0 1]
[1 1]
[2, 1]
[0 0]
iterx
[0 1]
[2 2]
[2, 0]
[0 0]
iterx
[0 1]
[1 1]
[0, 1]
[0 0]
iterx
[0 1]
[1 1]
[1, 0]
[0 0]
iterx
[0 1]
[2 2]
[2, 2]
[0 0]
iterx
[0 1]
[2 2]
[1, 0]
[0 0]
iterx
[0 1]
[3 3]
[2, 0]
[0 0]
iterx
[0 1]
[4 4]
[2, 1]
[0 0]
iterx
[0 1]
[3 3]
[0, 1]
[0 0]
iterx
[0 1]
[4 4]
[2, 2]
[0 0]
iterx
[0 1]
[3 3]
[0, 1]
[0 0]
iterx
[0 1]
[4 4]
[2, 0]
[0 0]
iterx
[0 1]
[3 3]
[0, 1]
[0 0]
iterx
[0 1]
[2 2]
[0, 1]
[0 0]
iterx
[0 1]
[3 3]
[2, 0]
[0 0]
iterx
[0 1]
[2 2]
[0, 0]
[0 0]
iterx
[0 1]
[3 3]
[2, 0]
[0 0]
iterx
[0 1]
[2 2]
[0, 2]
[0 0]
iterx
[0 1]
[2 2]
[1, 0]
[0 0]
iterx
[0

iterx
[0 1]
[2 2]
[2, 2]
[0 0]
iterx
[0 1]
[3 3]
[2, 2]
[0 0]
iterx
[0 1]
[4 4]
[2, 1]
[0 0]
iterx
[0 1]
[4 4]
[1, 1]
[0 0]
iterx
[0 1]
[3 3]
[0, 1]
[0 0]
iterx
[0 1]
[3 3]
[1, 2]
[0 0]
iterx
[0 1]
[3 3]
[1, 0]
[0 0]
iterx
[0 1]
[2 2]
[0, 1]
[0 0]
iterx
[0 1]
[3 3]
[2, 0]
[0 0]
iterx
[0 1]
[3 3]
[1, 2]
[0 0]
iterx
[0 1]
[2 2]
[0, 2]
[0 0]
iterx
[0 1]
[2 2]
[1, 1]
[0 0]
iterx
[0 1]
[2 2]
[1, 2]
[0 0]
iterx
[0 1]
[3 3]
[2, 0]
[0 0]
iterx
[0 1]
[3 3]
[1, 0]
[0 0]
iterx
[0 1]
[3 3]
[1, 1]
[0 0]
iterx
[0 1]
[3 3]
[1, 0]
[0 0]
iterx
[0 1]
[2 2]
[0, 0]
[0 0]
iterx
[0 1]
[1 1]
[0, 1]
[0 0]
iterx
[0 1]
[1 1]
[1, 2]
[0 0]
iterx
[0 1]
[0 0]
[0, 0]
[0 0]
iterx
[0 1]
[1 1]
[2, 2]
[0 0]
iterx
[0 1]
[0 0]
[0, 1]
[0 0]
iterx
[0 1]
[0 0]
[1, 2]
[0 0]
iterx
[0 1]
[0 0]
[0, 2]
[0 0]
iterx
[0 1]
[1 1]
[2, 0]
[0 0]
iterx
[0 1]
[2 2]
[2, 2]
[0 0]
iterx
[0 1]
[3 3]
[2, 2]
[0 0]
iterx
[0 1]
[4 4]
[2, 2]
[0 0]
iterx
[0 1]
[3 3]
[0, 2]
[0 0]
iterx
[0 1]
[2 2]
[0, 0]
[0 0]
iterx
[0 1]
[1 1]
[0, 2]
[0 0]
iterx
[0

KeyboardInterrupt: 

In [62]:
environment.is_vectorizable()

True

In [66]:
parallel_index, states = environment.reset(num_parallel=num_parallel)
actions = agent.act(parallel=parallel_index, states=states)


In [67]:
actions

[2, 1]

In [68]:
next_parallel, states, terminal, reward = environment.execute(actions=actions)

# state, reward, terminal_state = environment.execute(action)
# agent.observe(state, action, reward, terminal_state)

In [70]:
next_parallel, states, terminal, reward

(array([0, 1]), array([7, 3]), array([0, 0]), array([ 0.4, -0.4]))

In [72]:
agent.observe(terminal=terminal, reward=reward, parallel=parallel_index)

0

In [78]:
agent.close()
environment.close()

In [83]:
reward

array([-0.4])